In [69]:
!pip install eng_to_ipa
import eng_to_ipa as ipa

def is_vowel(character):

    ipa_vowels = "aeiou"
    return character in ipa_vowels

def is_whitespace(character):
    return character==' ' or character=='\n'

def get_phonetic_transcription(lyrics):
    lines = lyrics.splitlines()
    phonetic_lines = [ipa.convert(line) for line in lines]
    phonetic = "\n".join(phonetic_lines)

    return phonetic

import re
import numpy as np

def syllable_similarity(bar1, bar2):
    similarity = abs(syllables.estimate(bar1) - syllables.estimate(bar2))
    return similarity

def get_syllable_count_difference(lyrics):
    lines = lyrics.splitlines()
    i = 0
    similarity = 0
    while i < len(lines)-1:
        similarity += syllable_similarity(lines[i], lines[i+1])
        i += 2
    return similarity/len(lines)

def get_rhyme_density(lyrics, lookback=15):
    bars = SongLyrics(lyrics, lookback)
    return bars.avg_rhyme_length

def get_longest_rhyme(lyrics, lookback=15):
    bars = SongLyrics(lyrics, lookback)
    return bars.get_longest_rhyme()[0]

def get_unique_words(lyrics):
    words = lyrics.split()
    unique_words = set(words)
    return len(unique_words)/len(words)

def print_lyrics_stats(lyrics, lookback=15, artist=None, title=None):
    bars = SongLyrics(lyrics, lookback, artist, title)
    print(bars.title)
    print('------------------------------------------')
    print("Average rhyme length: %.3f\n" % bars.avg_rhyme_length)
    bars.print_rhyme(bars.longest_rhyme)
    print("Average syllable count difference: %.3f\n" % get_syllable_count_difference(lyrics))
    print("Percentage unique words: %.3f\n" % get_unique_words(lyrics))



class SongLyrics:
    '''
    This class is used to store and preprocess rap lyrics and calculate
    statistics like average rhyme length out of the lyrics.
    '''

    def __init__(self, text=None, lookback=15, artist=None, title=None):
        '''
        Lyrics can be read from the file (default) or passed directly
        to this constructor.
        '''
        self.raw_text = None
        # How many previous words are checked for a rhyme.
        self.lookback = lookback
        self.raw_text = text
        if artist == None or title == None:
            self.title = "Generated Song"
        else:
            self.title = title + " by " + artist

        if self.raw_text is not None:
            cleaning_ok = self.clean_text(self.raw_text)
            self.compute_vowel_representation()
            self.avg_rhyme_length, self.longest_rhyme = self.calculate_rhyme_stats()

    def clean_text(self, text):
        '''
        Preprocess text by removing unwanted characters and duplicate rows.
        '''

        self.text = text
        # If there are more than 2 consecutive newlines, remove some of them
        # (just to make the cleaned text look prettier)
        self.text = re.sub('\n\n+', '\n\n', self.text)
        # Remove duplicate rows
        self.lines = self.text.split('\n')

        unique_lines = set()
        new_text = ''
        for line in self.lines:
            line = line.strip()
            if len(line) > 0 and line in unique_lines:
                continue
            # Remove lines that are within brackets/parenthesis
            if len(line) >= 2 and ((line[0]=='[' and line[-1]==']') or (line[0]=='(' and line[-1]==')')):
                continue
            unique_lines.add(line)
            new_text += line + '\n'

        self.text = new_text

    def compute_vowel_representation(self):
        '''
        Compute a representation of the lyrics where only vowels are preserved.
        '''
        self.vowels_only = [] # Lyrics with all but vowels removed
        self.vowel_indices = [] # Indices of the vowels in self.text list
        self.word_end_indices = [] # Indices of the last characters of each word
        self.words = [] # List of words in the lyrics
        self.line_indices = []

        self.original_text = self.text
        self.text = get_phonetic_transcription(self.text)
        self.original_word_end_indices = []
        self.original_words = []

        previous_space_index = -1 # Index of the previous space char
        line_index = 0 # Line index of the current character
        # Go through the lyrics char by char
        for i in range(len(self.text)):
            self.line_indices.append(line_index)
            character = self.text[i]
            if is_vowel(character):
                # Ignore double vowels
                if i > 0 and self.text[i-1] == character:
                    # Index of a double vowel points to the latter occurrence
                    self.vowel_indices[-1] = i
                    continue
                self.vowels_only.append(character)
                self.vowel_indices.append(i)
            elif is_whitespace(character):
                if character in '\n':
                    line_index += 1
                elif character in '.!?' and i < len(self.text)-1 and self.text[i+1] != '\n':
                    line_index += 1
                if len(self.vowels_only) > 0 and not is_whitespace(self.text[i-1]):
                    new_word = self.text[previous_space_index+1:self.vowel_indices[-1]+1]
                    no_vowels = True
                    for char in new_word:
                        if is_vowel(char):
                            no_vowels = False
                            break
                    if no_vowels:
                        previous_space_index = i
                        continue
                    self.word_end_indices.append(len(self.vowels_only)-1)
                    self.words.append(new_word)
                previous_space_index = i

        self.original_lines = self.original_text.split('\n')

    def rhyme_length(self, word_position_2):
        '''
        Length of rhyme (in vowels). The latter part of the rhyme ends with
        word self.words[word_position_2].

        Input:
            word_position_2       Word index of the end of the rhyme.
        '''
        max_length = 0
        max_word_position_1 = None
        word_position_1 = max(0, word_position_2 - self.lookback)
        while word_position_1 < word_position_2:
            rhyme_length = self.fixed_rhyme_length(word_position_1, word_position_2)
            if rhyme_length > max_length:
                max_length = rhyme_length
                max_word_position_1 = word_position_1
            word_position_1 += 1
        return max_length, max_word_position_1

    def fixed_rhyme_length(self, word_position_1, word_position_2):
        '''
        Length of rhyme (in vowels). The first part of the rhyme ends with
        self.words[word_position_1] and the latter part with word self.words[word_position_2].

        Input:
            word_position_1       Word index of the last word in the first part of the rhyme.
            word_position_2       Word index of the end of the rhyme.
        '''
        if word_position_1 < 0:
            return 0
        elif self.words[word_position_1] == self.words[word_position_2]:
            return 0
        vowel_index_1 = self.word_end_indices[word_position_1]
        vowel_index_2 = self.word_end_indices[word_position_2]
        length = 0
        while self.vowels_only[vowel_index_1 - length] == self.vowels_only[vowel_index_2 - length]:
            if word_position_1 > 0 and vowel_index_1 - length <= self.word_end_indices[word_position_1 - 1] and word_position_2 > 0 and vowel_index_2 - length <= self.word_end_indices[word_position_2 - 1]:
                prev_start_index_1 = self.vowel_indices[vowel_index_1 - length]
                while prev_start_index_1 > 0 and not is_whitespace(self.text[prev_start_index_1 - 1]):
                    prev_start_index_1 -= 1
                prev_start_index_2 = self.vowel_indices[vowel_index_2 - length]
                while prev_start_index_2 > 0 and not is_whitespace(self.text[prev_start_index_2 - 1]):
                    prev_start_index_2 -= 1
                next_start_index_1 = self.vowel_indices[vowel_index_1 - length]
                while next_start_index_1 < len(self.text) - 1 and not is_whitespace(self.text[next_start_index_1 + 1]):
                    next_start_index_1 += 1
                next_start_index_2 = self.vowel_indices[vowel_index_2 - length]
                while next_start_index_2 < len(self.text) - 1 and not is_whitespace(self.text[next_start_index_2 + 1]):
                    next_start_index_2 += 1
                if next_start_index_1 - prev_start_index_1 == next_start_index_2 - prev_start_index_2 and self.text[prev_start_index_1:next_start_index_1 + 1] == self.text[prev_start_index_2:next_start_index_2 + 1]:
                    break
            length += 1
            if vowel_index_1 - length < 0 or vowel_index_2 - length <= vowel_index_1:
                break
        if length == 1:
            length = 0
        return length

    def calculate_rhyme_stats(self):
        '''
        Compute the average rhyme length of the song and the longest rhyme.

        Output:
            Average rhyme length (float)
            Longest rhyme which is a 3-tuple with:
                (length, word index of the first part of the rhyme,
                         word index of the latter part of the rhyme)
        '''
        rhyme_lengths = []
        max_rhyme = (0, None, None)
        for word_position_2 in range(1, len(self.word_end_indices)):
            rhyme_length, word_position_1 = self.rhyme_length(word_position_2)
            rhyme_lengths.append(rhyme_length)
            if rhyme_length > max_rhyme[0]:
                max_rhyme = (rhyme_length, word_position_1, word_position_2)
        rhyme_lengths = np.array(rhyme_lengths)
        if len(rhyme_lengths) > 0:
            avg_rhyme_length = np.mean(rhyme_lengths)
        else:
            avg_rhyme_length = 0
        return avg_rhyme_length, max_rhyme

    def get_avg_rhyme_length(self):
        return self.avg_rhyme_length

    def print_song_stats(self):
        print('------------------------------------------')
        print("Avg rhyme length: %.3f\n" % self.avg_rhyme_length)

        self.print_rhyme(self.longest_rhyme)

    def print_rhyme(self, rhyme_tuple):
        print(self.get_rhyme_str(rhyme_tuple))

    def get_rhyme_str(self, rhyme_tuple):
        ret = ''
        rhyme_length, word_position_1, word_position_2 = rhyme_tuple
        if word_position_1 is None or word_position_2 is None:
            return ''
        vowel_index_2 = self.vowel_indices[self.word_end_indices[word_position_2]]
        vowel_index_2_original = vowel_index_2
        while not is_whitespace(self.text[vowel_index_2]):
            vowel_index_2 += 1
        vowel_index_1 = self.vowel_indices[self.word_end_indices[word_position_1] - rhyme_length]
        vowel_index_1_original = vowel_index_1
        while self.text[vowel_index_1] != '\n' and vowel_index_1 > 0:
            vowel_index_1 -= 1

        capitalized_line = ''
        rhyming_vowels_1, rhyming_vowels_2 = self.get_rhyming_vowels(rhyme_tuple)
        for i in range(vowel_index_1, vowel_index_2 + 1):
            if i == min(rhyming_vowels_1) or i == min(rhyming_vowels_2):
                capitalized_line += ' | ' + self.text[i]
            elif i == max(rhyming_vowels_1) or i == max(rhyming_vowels_2):
                capitalized_line += self.text[i] + '|'
            else:
                capitalized_line += self.text[i]
        ret += "Longest rhyme (l=%d): %s\n" % (rhyme_length, capitalized_line)
        line_begin = self.line_indices[vowel_index_1]
        line_end = self.line_indices[vowel_index_2]
        for i in range(line_begin, line_end + 1):
            if i < len(self.original_lines):
                ret += self.original_lines[i] + '\n'
        return ret

    def get_longest_rhyme(self):
        rhyme_str = self.get_rhyme_str(self.longest_rhyme)
        return self.longest_rhyme[0], rhyme_str

    def get_rhyming_vowels(self, rhyme_tuple):
        rhyme_length, word_position_1, word_position_2 = rhyme_tuple
        if word_position_1 is None or word_position_2 is None:
            return ([-1], [-1])

        rhyming_vowels_1 = []
        count_caps = 0
        index = self.vowel_indices[self.word_end_indices[word_position_1]]
        while count_caps < rhyme_length:
            if is_vowel(self.text[index]):
                rhyming_vowels_1.append(index)
                if self.text[index] != self.text[index + 1]:
                    count_caps += 1
            index -= 1

        rhyming_vowels_2 = []
        count_caps = 0
        index = self.vowel_indices[self.word_end_indices[word_position_2]]
        last_index = index
        while count_caps < rhyme_length:
            if is_vowel(self.text[index]):
                rhyming_vowels_2.append(index)
                if index == last_index or self.text[index] != self.text[index + 1]:
                    count_caps += 1
            index -= 1

        return (rhyming_vowels_1, rhyming_vowels_2)


In [36]:

import numpy as np
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import bigrams, ngrams, everygrams
from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends
from nltk.lm import MLE, KneserNeyInterpolated, Lidstone, Laplace, AbsoluteDiscountingInterpolated


In [37]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [38]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df
train_df.rename({"Lyrics": "lyric", "Title": "song", "Artist": "artist"}, axis=1, inplace=True)
test_df.rename({"Lyrics": "lyric", "Title": "song", "Artist": "artist"}, axis=1, inplace=True)

## Tokenization

Default word tokenizer removes new line symbols.

In [39]:
# # Test
# lyric = [word_tokenize(line) + ['\n'] for line in train['lyric'][0].split('\n')]
# flat_list = list(itertools.chain(*lyric))[:-1]  # remove last new line symbol

In [40]:
def tokenize(dataset):
    return [
        nltk.flatten(([
            word_tokenize(line) + ['\n']
            for line in lyric.split('\n')
        ]))[:-1]
        for lyric in list(dataset['lyric'])
        if not isinstance(lyric, float)  # remove nan values
    ]

tokenized_train = tokenize(train_df)
tokenized_test = tokenize(test_df)

## Setup and stats

In [41]:
N = 3

In [42]:
def generate_ngrams(dataset, every=False):
    if every:
        return [list(everygrams(pad_both_ends(example, n=N), max_len=N)) for example in dataset]
    else:
        return [list(ngrams(pad_both_ends(example, n=N), n=N)) for example in dataset]

train_ngrams = generate_ngrams(tokenized_train, every=True)
test_ngrams = generate_ngrams(tokenized_test)

In [43]:
def generate_vocabulary(dataset):
    return list(nltk.lm.preprocessing.flatten(pad_both_ends(example, n=N) for example in dataset))

train_vocabulary = generate_vocabulary(tokenized_train)
test_vocabulary = generate_vocabulary(tokenized_test)

In [44]:
print("Training set contains", len(set(train_vocabulary)), "distinct tokens.")
print("Test set contains", len(set(test_vocabulary)), "distinct tokens.")
print("Test set has", len(set(test_vocabulary) - set(train_vocabulary)), "tokens not in train test.")

Training set contains 6757 distinct tokens.
Test set contains 3578 distinct tokens.
Test set has 1148 tokens not in train test.


In [45]:
from collections import Counter
Counter(train_vocabulary).most_common(10)

[('\n', 8628),
 (',', 5959),
 ('I', 3145),
 ('the', 2011),
 ("'", 1494),
 ('you', 1471),
 ('(', 1468),
 (')', 1468),
 ('a', 1204),
 ('it', 1164)]

## Training

In [46]:
lm = Lidstone(1e-4, N)
lm.fit(train_ngrams, train_vocabulary)

In [47]:
len(lm.vocab)

6758

## evaluation metrics


### Perplexity (PPL)

In [48]:
lm.perplexity(nltk.lm.preprocessing.flatten(train_ngrams))

34.729352351396486

In [49]:
lm.perplexity(nltk.lm.preprocessing.flatten(test_ngrams))

743.7346299905059

### Generate songs after first line

In [50]:
re.sub(r'[ ]+\n[ ]+', r'\n', ' '.join(lm.generate(100, text_seed=['I', 'found'])))

'\n Oh yeah , yeah )\nIf I do , I get it\n\n Girl I perform for ya\n\n I did ( ayy )\n... </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>'

In [51]:
first_lines = [song[:song.index('\n')+1] for song in tokenized_test if '\n' in song]
len(first_lines)

110

In [52]:
first_lines[0]

['(', 'Ugh', ',', 'you', "'re", 'a', 'monster', ')', '\n']

In [70]:
results = []
for first_line in first_lines:
    result = ' '.join(first_line)[:-2] + '\n'
    result += re.sub(r'[ ]+\n[ ]+', r'\n', ' '.join(lm.generate(100, text_seed=first_line, random_seed=0)))
    result = re.sub(' </s>.*', '', result)
    results.append(result)

In [54]:
print(results[8])

Traphouse Mob
Yoz , what , what I pay for hoes ( yeah )
Shawty go jogging every morning )
Bum bum be-dum bum ( I ca n't stress about no bitch 'cause I know
Ooh , that I 'd like to fuck I look around , niggas spinnin '

 Guess we all Pisces
Your last one standin ' in my hallways , I get down
All the ladies , they love smoking and love drinking , some love deep inside of the storm when it feels like it when you turn the music up


### Rhyme Density (RD)

In [71]:

def get_rhyme_density(lyrics, lookback=15):
    bars = SongLyrics(lyrics, lookback)
    return bars.avg_rhyme_length
results_rd = np.array([get_rhyme_density(song) for song in results])

In [72]:
round(np.mean(results_rd),2)

0.75

In [73]:
round(np.std(results_rd),2)

0.26

### Unique Words (UW)

In [74]:
results_uw = np.array([get_unique_words(song) for song in results])

In [75]:
print(round(np.mean(results_uw),2), round(np.std(results_uw),2))

0.74 0.07
